In [1]:
%pwd

'/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/Assignment-4'

In [1]:
import os
import re
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments
from transformers.trainer_utils import IntervalStrategy, SaveStrategy
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
import evaluate
from sentence_transformers import SentenceTransformer, util
from sklearn.model_selection import train_test_split
import mlflow
import json

/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/.nlp-genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# df=pd.read_excel("Table Insights/table_insights_labeled_data.xlsx")
# df.rename(columns={df.columns[0]: "device"}, inplace=True)
# df

,device,date_,cpu_usage,ram_usage,diskio_usage,question_0,insight_0,question_1,insight_1,question_2,insight_2,serialnumber_org
0,Device_1,2022-08-02,7.845169,52.662081,NaN,What does the data contains and say about. Res...,The data contains information about a computer...,What are the top 3 most important insights,1. The highest CPU usage was recorded on 2022-...,What are the top 3 abberations present in the ...,1. The diskio_usage column has no data (NaN) f...,1BBZ4Y2
1,Device_1,2022-08-03,5.029416,53.519746,NaN,0,0,0,0,0,0,1BBZ4Y2
2,Device_1,2022-08-04,4.855019,53.656122,NaN,0,0,0,0,0,0,1BBZ4Y2
3,Device_1,2022-08-05,8.014844,55.245124,NaN,0,0,0,0,0,0,1BBZ4Y2
4,Device_1,2022-08-07,16.909919,54.216115,NaN,0,0,0,0,0,0,1BBZ4Y2
...,...,...,...,...,...,...,...,...,...,...,...,...
5276,Device_215,2022-05-25,NaN,NaN,90.005963,0,0,0,0,0,0,GUI43W95GE
5277,Device_215,2022-05-26,NaN,NaN,75.829685,0,0,0,0,0,0,GUI43W95GE
5278,Device_215,2022-05-27,NaN,NaN,88.540792,0,0,0,0,0,0,GUI43W95GE
5279,Device_215,2022-05-28,NaN,NaN,77.175451,0,0,0,0,0,0,GUI43W95GE


In [ ]:
# summarizer=pipeline("text2text-generation",model="google/flan-t5-base")

# #summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [ ]:
# PROMPTS = [
#     "Summarize the dataset focusing on CPU, RAM, and Disk performance.",
#     "Write a short report explaining the system behavior over time.",
#     "Generate business-level insights based on device performance trends.",
#     "Summarize any anomalies or outliers in the dataset.",
#     "Create a technical summary of how system resources were used."
# ]

In [ ]:
# def table_to_text(row):
    # return f"Device {row['device']} had CPU usage {row['cpu_usage']:.2f}% and RAM usage {row['ram_usage']:.2f}% on {row['date_']}."


In [ ]:
# texts=[table_to_text(row) for _,row in df.iterrows()]

In [ ]:
# input_text=" ".join(texts)

In [ ]:
# summary=summarizer(f"Summarize this device usage data and provide 3 business insights:\n{input_text}")

In [ ]:
# print("Generated Summary:\n:",summary[0]['generated_text'])

Generated Summary:
: Device Device_1 had CPU usage 8.01% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 4.86% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 8.01% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 16.91% and RAM usage 54.22% on 2022-08-05 00:00:00. Device Device_1 had CPU usage 8.01% and RAM usage 53.52% on 2022-08-07 00:00:00. Device Device_1 had CPU usage 6.91% and RAM usage 62.33% on 2022-08-10 00:00:00. Device Device_1 had CPU usage 7.50% and RAM usage 63.40% on 2022-08-11 00:00:00. Device


In [ ]:
# prompt_1 = f"Summarize CPU and RAM usage trends and identify anomalies:\n{input_text}"
# prompt_2 = f"Provide actionable business insights from the following machine usage table:\n{input_text}"
# prompt_3 = f"Write a concise report on system performance trends and abnormalities:\n{input_text}"

# for i, prompt in enumerate([prompt_1, prompt_2, prompt_3], 1):
#     result = summarizer(prompt)
#     print(f"\n🔹 Prompt {i} Result:\n", result[0]['generated_text'])



🔹 Prompt 1 Result:
 Device Device_1 had CPU usage 5.03% and RAM usage 52.66% on 2022-08-02 00:00:00. Device Device_1 had CPU usage 4.86% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 8.01% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 16.91% and RAM usage 55.25% on 2022-08-05 00:00:00. Device Device_1 had CPU usage 8.01% and RAM usage 55.32% on 2022-08-07 00:00:00. Device Device_1 had CPU usage 6.97% and RAM usage 59.46% on 2022-08-09 00:00:00. Device Device_1 had CPU usage 6.91% and RAM usage 62.33% on 2022-08-10 00:

🔹 Prompt 2 Result:
 Device Device_1 had CPU usage 7.81% and RAM usage 66.83% on 2022-08-21 00:00:00.

🔹 Prompt 3 Result:
 Device Device_1 had CPU usage 8.01% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 4.86% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 4.86% and RAM usage 53.52% on 2022-08-04 00:00:00. Device Device_1 had CPU usage 8.01% and RAM usage 5

In [ ]:
# def get_random_sample(df, frac=0.4):
#     return df.sample(frac=frac, random_state=None)

In [ ]:
# def summarize_table(df,pt,temperature=0.8):
#     prompt = pt
#     sample_df = get_random_sample(df)
#     context = sample_df.head(5).to_string(index=False)

#     full_input = f"""
#     You are a data analyst.
#     The following is a table showing device usage statistics.
#     Based on the data, {prompt}

#     Table:
#     {context}
#     """

#     output = summarizer(full_input, max_length=150, min_length=40, do_sample=True, temperature=temperature)[0]['generated_text']

#     print(f"🧾 Prompt: {prompt}\n")
#     print("📊 Summary:\n", output)
#     return output

In [ ]:
# for i in PROMPTS:
#     summary = summarize_table(df,pt=i,temperature=0.9)

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧾 Prompt: Summarize the dataset focusing on CPU, RAM, and Disk performance.

📊 Summary:
 Device_42 2023-03-17 12.971891 62.290418 5.299380 0 0 0 0 0 5CD127F945 Device_38 2022-08-31 2.259321 10.787057 NaN 0 0 0 0 0 0 0 4CE9241B5C Device_112 2023-03-27 11.216762 72.827133 1.949365 0 0 0 0 0 0 0 0 7O2JJQMD8J Device_147 2022-04-30 34.880517 NaN 0 0 0 0 0 0 0 0 7O2JJQMD8J


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Token indices sequence length is longer than the specified maximum sequence length for this model (661 > 512). Running this sequence through the model will result in indexing errors


🧾 Prompt: Write a short report explaining the system behavior over time.

📊 Summary:
 Device_48 2022-02-01 NaN 11.721103 NaN 0 0 0 0 0 0 0 74KG71NYHC Device_21 2022-05-17 NaN NaN 89.824853 0 0 0 0 0 0 0 0 E6VESLF8UI Device_191 2022-06-10 5.888569 48.679107 NaN 0 0 0 0 0 0 0 0 MXL9233F04 Device_93 2022-07-02 NaN NaN 16.780572 0 0 0 0 0 0 0 0 R5FSIKUTS6


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧾 Prompt: Generate business-level insights based on device performance trends.

📊 Summary:
 5CD0139WZ2 CPU Usage Variability: CPU Usage varies significantly across the dataset, ranging from 6.75% to 20.15%.nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧾 Prompt: Summarize any anomalies or outliers in the dataset.

📊 Summary:
 Device_6 2023-03-05 63.617553 48.875857 1.339953 0 0 0 0 0 0 MXL0404FWG Device_26 2022-08-04 30.964149 84.776425 NaN 0 0 0 0 0 0 5CG839181R Device_83 2023-03-25 7.782140 69.247028 1.037882 0 0 0 0 0 0 0 5CG1092DC0 Device_212 2023-03-16 9.754410 81.015184 5.004808 0 0 0 0 0 0 0 ZK016729U


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧾 Prompt: Create a technical summary of how system resources were used.

📊 Summary:
 Device_160 2022-08-23 24.701188 76.204835 NaN 0 0 0 0 0 0 5CG8224YM4 Device_174 2023-03-22 6.521767 79.398007 1.833206 0 0 0 0 0 0 0 9T688EZ74Y Device_181 2022-10-14 NaN NaN 70.233007 0 0 0 0 0 0 0 9T688EZ74Y Device_61 2022-05-27 15.085230 NaN NaN 0 0 0 0 0 0 0 62NVDZ3HEK Device_122 2023-03-13 46.341837 49.944305 28.456247 0 0 0 0 0 0 0 5CG2213810


In [5]:
# Configuration
MODEL_NAME = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
MAX_SEQ_LENGTH = 1024
LORA_R = 16
LORA_ALPHA = 32
NUM_TRAIN_EPOCHS = 3
LR = 2e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Initialize MLflow Tracking
# We use a local tracking URI for Colab
MLFLOW_TRACKING_URI = "mlruns"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("IT-Giant_Table-to-Insights")

Using device: cpu


<Experiment: artifact_location='/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/Assignment-4/mlruns/308567176882587203', creation_time=1762598374337, experiment_id='308567176882587203', last_update_time=1762598374337, lifecycle_stage='active', name='IT-Giant_Table-to-Insights', tags={}>

In [6]:
#DATA LOADING, SERIALIZATION, AND SPLIT

FILE_PATH = "Table Insights/table_insights_labeled_data.xlsx"
df_raw = pd.read_excel(FILE_PATH)

# Clean Column Names and Create Group IDs
new_columns = list(df_raw.columns)
new_columns[0] = 'device_log_group'
df_raw.columns = new_columns
df_raw.rename(columns={'serialnumber_org': 'device_serial'}, inplace=True)
df_raw['group_id'] = df_raw['device_log_group'].ffill() + '_' + df_raw['device_serial'].astype(str)

# Serialization Function (As defined previously)
def group_and_serialize_logs(group):
    target_row = group[(group['insight_0'] != '0') & (group['insight_0'] != 0)].head(1)
    if target_row.empty:
        return pd.Series({'input_prompt': None, 'target_insight': None})

    target_insight = target_row['insight_0'].iloc[0]
    kpi_cols = ['cpu_usage', 'ram_usage', 'diskio_usage']
    agg_data = group[kpi_cols].agg(['mean', 'max', 'count']).to_dict()
    null_counts = group[kpi_cols].isnull().sum().to_dict()

    prompt_template = f"""
    Analyze the following machine usage log data for Device: {group['device_serial'].iloc[0]}.
    The data spans {agg_data['cpu_usage']['count']} data points.

    ### Key Performance Indicators (KPIs):
    - CPU Usage (Mean/Max): {agg_data['cpu_usage']['mean']:.2f}% / {agg_data['cpu_usage']['max']:.2f}%
    - RAM Usage (Mean/Max): {agg_data['ram_usage']['mean']:.2f}% / {agg_data['ram_usage']['max']:.2f}%

    ### Data Completeness:
    - Disk IO Usage: {agg_data['diskio_usage']['count']} non-null entries.
    - Missing CPU entries: {null_counts['cpu_usage']}

    TASK: Generate a concise, actionable business insight (similar to the provided GPT insight_0) based on this summary.
    """
    return pd.Series({'input_prompt': prompt_template.strip(), 'target_insight': target_insight})

# Apply Grouping
df_processed = df_raw.groupby('group_id').apply(group_and_serialize_logs).reset_index()
df_processed.dropna(subset=['target_insight'], inplace=True)
df_processed.rename(columns={'input_prompt': 'text', 'target_insight': 'summary'}, inplace=True)

# Train/Test Split
train_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
test_df_for_eval = test_df.copy()

# Convert to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['text', 'summary']])
test_dataset = Dataset.from_pandas(test_df_for_eval[['text', 'summary']])
print(train_dataset.column_names)

['text', 'summary', '__index_level_0__']


/tmp/ipykernel_175444/2514243728.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_processed = df_raw.groupby('group_id').apply(group_and_serialize_logs).reset_index()


In [ ]:
#BASE MODEL INFERENCE AND BASELINE LOGGING


with mlflow.start_run(run_name="Base_Model_ZeroShot") as run:
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("tuning_method", "None (Zero-Shot)")
    mlflow.log_param("max_seq_length", MAX_SEQ_LENGTH)


    base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16, device_map="auto")
    base_model.to(DEVICE)

    def generate_insight(prompt_text, model, tokenizer, max_length=150):
        input_text = f"Summarize machine usage logs into actionable business insights. INPUT: {prompt_text}"
        inputs = tokenizer(input_text, return_tensors="pt", max_length=MAX_SEQ_LENGTH, truncation=True).to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=max_length, num_beams=4, do_sample=False, early_stopping=True)
        return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Generate Predictions
    base_model_predictions = [generate_insight(prompt, base_model, tokenizer) for prompt in test_df_for_eval['text'].tolist()]
    test_df_for_eval['base_prediction'] = base_model_predictions
    references = test_df_for_eval['summary'].tolist()

    # Evaluation
    rouge_metric = evaluate.load("rouge")
    bleu_metric = evaluate.load("bleu")
    semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

    def calculate_semantic_match(predictions, references, model):
        pred_embeddings = model.encode(predictions, convert_to_tensor=True)
        ref_embeddings = model.encode(references, convert_to_tensor=True)
        cos_scores = util.cos_sim(pred_embeddings, ref_embeddings)
        return np.mean([cos_scores[i, i].item() for i in range(len(predictions))])

    base_rouge = rouge_metric.compute(predictions=base_model_predictions, references=references)
    base_bleu = bleu_metric.compute(predictions=base_model_predictions, references=references, max_order=4)
    base_semantic = calculate_semantic_match(base_model_predictions, references, semantic_model)

    # Log Metrics to MLflow
    mlflow.log_metric("base_bleu_4", round(base_bleu['bleu'], 4))
    mlflow.log_metric("base_rouge_l", round(base_rouge['rougeL'], 4))
    mlflow.log_metric("base_semantic_match", round(base_semantic, 4))
    print("Base Model Metrics Logged to MLflow.")

    # Save the base model results for comparison
    test_df_for_eval[['text', 'summary', 'base_prediction']].to_csv("base_model_predictions.csv", index=False)
    mlflow.log_artifact("base_model_predictions.csv")

Base Model Metrics Logged to MLflow.


In [ ]:
#LORA FINE-TUNING AND MLFLOW LOGGING


with mlflow.start_run(run_name="LoRA_FineTuned_FlanT5") as run:
    # 4.1 Log Parameters
    mlflow.log_param("model_name", MODEL_NAME)
    mlflow.log_param("tuning_method", "LoRA PEFT")
    mlflow.log_param("lora_r", LORA_R)
    mlflow.log_param("lora_alpha", LORA_ALPHA)
    mlflow.log_param("num_epochs", NUM_TRAIN_EPOCHS)
    mlflow.log_param("learning_rate", LR)

    def _fix_json_serialization(obj):
        if isinstance(obj, set):
            return list(obj)
        if isinstance(obj, tuple):
            return list(obj)
        raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

    # 4.2 Data Formatting for SFT
    def format_for_sft(example):
        prompt = f"Summarize machine usage logs into actionable business insights. INPUT: {example['text']}"
        target = example['summary']
        return {"text": f"{prompt} TARGET: {target}{tokenizer.eos_token}"}

    cols_to_remove = ['summary', '__index_level_0__']
    train_cols = train_dataset.column_names
    test_cols = test_dataset.column_names

    # Filter the list to only include existing columns
    cols_to_remove_train = [col for col in cols_to_remove if col in train_cols]
    cols_to_remove_test = [col for col in cols_to_remove if col in test_cols]

    # Apply the map function using the filtered list
    print(f"Columns to remove from train_dataset: {cols_to_remove_train}")
    print(f"Columns to remove from test_dataset: {cols_to_remove_test}")

    train_dataset = train_dataset.map(format_for_sft, remove_columns=['summary', '__index_level_0__'])
    test_dataset = test_dataset.map(format_for_sft, remove_columns=['summary', '__index_level_0__'])

    # 4.3 Configure and Apply LoRA
    lora_config = LoraConfig(
        r=LORA_R, lora_alpha=LORA_ALPHA, target_modules=["q", "v"],
        lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM",
    )
    # Log the full config as a JSON artifact
    with open("lora_config.json", "w") as f:
        json.dump(lora_config.to_dict(), f,default=_fix_json_serialization)
    mlflow.log_artifact("lora_config.json")

    model_for_tuning = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16, device_map="auto")
    model_for_tuning = get_peft_model(model_for_tuning, lora_config)

    # 4.4 Training Arguments and SFTTrainer
    training_args = TrainingArguments(
    output_dir="./lora_results",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=4,
    learning_rate=LR,
    logging_steps=10,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=100,
    fp16=True,
    load_best_model_at_end=True,
    report_to="mlflow"
)

    trainer = SFTTrainer(
        model=model_for_tuning,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        processing_class=tokenizer
    )

    print("\n--- Starting LoRA Fine-Tuning ---")
    trainer.train()

    # 4.5 Save Adapter and Log Model
    ADAPTER_PATH = "flan_t5_table_insights_adapter"
    trainer.model.save_pretrained(ADAPTER_PATH)
    tokenizer.save_pretrained(ADAPTER_PATH)

    # Log the adapter weights and tokenizer to MLflow
    mlflow.log_artifact(ADAPTER_PATH)
    print("Fine-Tuned Adapter Logged to MLflow Artifacts.")

Columns to remove from train_dataset: ['summary', '__index_level_0__']
Columns to remove from test_dataset: ['summary', '__index_level_0__']


Map: 100%|██████████| 43/43 [00:00<00:00, 8485.30 examples/s]
`torch_dtype` is deprecated! Use `dtype` instead!
Truncating eval dataset: 100%|██████████| 43/43 [00:00<00:00, 13263.35 examples/s]



--- Starting LoRA Fine-Tuning ---


/mnt/c/Users/sasidhar.chennup/Documents/Tiger-Training/NLP-GenAI-Classroom/NLP-Gen-AI-classroom/.nlp-genai/lib/python3.10/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


IndentationError: unexpected indent (3374530825.py, line 7)